In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/binary-classification-with-a-bank-dataset-clone/sample_submission.csv
/kaggle/input/binary-classification-with-a-bank-dataset-clone/train.csv
/kaggle/input/binary-classification-with-a-bank-dataset-clone/test.csv


In [12]:
import os

os.listdir("/kaggle/input")


['binary-classification-with-a-bank-dataset-clone']

In [13]:
import pandas as pd

train = pd.read_csv(
    "/kaggle/input/binary-classification-with-a-bank-dataset-clone/train.csv"
)

test = pd.read_csv(
    "/kaggle/input/binary-classification-with-a-bank-dataset-clone/test.csv"
)



In [14]:
target_col = "y"


In [15]:
X = train.drop(columns=[target_col])
y = train[target_col]

X_test = test.copy()


In [16]:
print(train.shape)
print(test.shape)


(750000, 18)
(250000, 17)


In [17]:
target_col = "y"

X = train.drop(columns=[target_col])
y = train[target_col]

X_test = test.copy()


In [18]:
X = X.drop(columns=["id"], errors="ignore")
X_test = X_test.drop(columns=["id"], errors="ignore")


In [19]:
X = pd.get_dummies(X)
X_test = pd.get_dummies(X_test)

X, X_test = X.align(X_test, join="left", axis=1, fill_value=0)


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [21]:
# Drop high-cardinality / heavy columns
cols_to_drop = ["job", "education", "month", "poutcome", "contact"]

X_reduced = X.drop(columns=cols_to_drop, errors="ignore")
X_test_reduced = X_test.drop(columns=cols_to_drop, errors="ignore")


In [22]:
X_reduced = pd.get_dummies(X_reduced)
X_test_reduced = pd.get_dummies(X_test_reduced)

X_reduced, X_test_reduced = X_reduced.align(
    X_test_reduced, join="left", axis=1, fill_value=0
)


In [23]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_reduced, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [24]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train, y_train)


GaussianNB()

In [25]:
from sklearn.metrics import roc_auc_score

val_probs = model.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, val_probs)


np.float64(0.8391366843565174)

In [26]:
model.fit(X_reduced, y)


GaussianNB()

In [27]:
test_probs = model.predict_proba(X_test_reduced)[:, 1]


In [28]:
submission = pd.read_csv(
    "/kaggle/input/binary-classification-with-a-bank-dataset-clone/sample_submission.csv"
)

submission["y"] = test_probs
submission.to_csv("submission.csv", index=False)


In [29]:
import pandas as pd

submission = pd.read_csv(
    "/kaggle/input/binary-classification-with-a-bank-dataset-clone/sample_submission.csv"
)

submission.head()


,id,y
0,750000,0.5
1,750001,0.5
2,750002,0.5
3,750003,0.5
4,750004,0.5


In [30]:
submission["y"] = test_probs
submission.head()


,id,y
0,750000,1.172108e-07
1,750001,6.483502e-01
2,750002,1.713832e-08
3,750003,5.661023e-11
4,750004,3.983328e-04


In [31]:
submission.to_csv("submission.csv", index=False)
